## Filtros de Kalman

El filtro de Kalman, aparecido a principios de la década de los sesenta, es de una importancia comparable
a los trabajos realizados por Nyquist y Bode en la década de los veinte y los de Wiener en los años treinta.
El filtro de Kalman permite estimar en tiempo real el vector de estado de un sistema dinámico lineal a partir
de medidas ruidosas indirectas que de él se van tomando. Estas estimaciones en tiempo real del estado del
sistema son valiosas en sí mismas cuando el sistema opera en lazo abierto; pero considerando la posibilidad
de operación en lazo cerrado, las mismas pueden ser utilizadas para sintetizar una acción de control
adecuada que lleve el sistema al estado deseado. [1]

- Filtro de kalman caso vectorial

> Nótese un sistema dado por la expresión:

$$x_k = f(x_{k-1}) + w_{k-1},\;\; w_{k-1}\sim \mathcal{N}(0, Q_{t})$$
$$z_k = h(x_k) + v_{k},\;\; v_{k-1}\sim \mathcal{N}(0, R_{t})$$

Donde 
- $x_k$ denota el vector de estados desconocidos en un tiempo k.
- La función $f$ es un operador de transición que mapea el espacio de estados recordemos que puede ser visto como:
$$\dot{x} = Ax + Bu + w$$
- Viendola como una ecuación en diferencias tendremos que
$$x[k+1] = Ax[k] + Bu + w[k]$$
- $z_{k}$ es el vector de observaciones
- Se asume que las variables del sistema tiene ruido con distribución normal con media 0 y varianza $Q_{t}$ y $R_{t}$, respectivamente.

> Tambien se toma como el estado inicial al vector $x_{0}$ un vector con media conocida $\mu_{0} = E[x_{0}]$ y covarianza $P_{0} = E[(x_{0}-\mu_{0})(x_{0}-\mu_{0})^{T}]$.

- Con $E[x]$ y $E[xy]$ representada como:

$$ \sum_{i=1}^{N}x_{i}P(X=x_{i})$$
$$ \sum_{i=1}^{N}\sum_{j=1}^{N} x_{i}y_{j}P(X=x_{i}, Y=y_{j})$$

- Para el caso del filtro de Kalman se cumple que:
$$E[w_{k}w_{k}^{T}] = Q_{t}$$
$$E[v_{k}v_{k}^{T}] = R_{t}$$

- Siendo estos errores no dependientes entre si, osea:

$$E[w_{k}v_{j}^{T}] = 0$$


In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.ion()

In [3]:
v1 = np.loadtxt('T-Bot_FilteredData.dat')
v1[np.where(v1[:,0]<0.001),0]=0.013

FileNotFoundError: T-Bot_FilteredData.dat not found.

In [4]:
angle = 0
filter_weighting = 0.04

def getAngleCFilter(pitch, gyro_rate, dt):
    global angle
    angle += gyro_rate * dt
    angle += filter_weighting * (pitch - angle)
    return angle

In [5]:
bias = 0
R_measure = 0.15 # measurement noise
Q_angle = 0.1 # process noise 
Q_bias = 0.3 # 
R_measure = 1 # measurement noise


# create arrays

P = np.zeros((2,2))
K = np.zeros(2)


def getAngle(pitch, gyrorate, dt):
    global bias
    global angle
    global P
    rate = gyrorate - bias;
    angle += dt * rate;
    # Update estimation error covariance - Project the error covariance ahead
    P[0,0] += dt * (dt*P[1,1] - P[0,1] - P[1,0] + Q_angle)
    P[0,1] -= dt * P[1,1]
    P[1,0] -= dt * P[1,1]
    P[1,1] += Q_bias * dt

    # Calculate Kalman gain - Compute the Kalman gain
    S = P[0,0] + R_measure
    K[0] = P[0,0] / S
    K[1] = P[1,0] / S
    y = pitch - angle

    #Calculate angle and bias - Update estimate with measurement zk (newAngle)
    angle += K[0] * y
    bias += K[1] * y

    #Calculate estimation error covariance - Update the error covariance
    P[0,0] -= K[0] * P[0,0]
    P[0,1] -= K[0] * P[0,1]
    P[1,0] -= K[1] * P[0,0]
    P[1,1] -= K[1] * P[0,1]
    return angle

In [ ]:
# https://www.klikrobotics.com/FilterTutorial.html
